In [ ]:
!pip install zipfile36
import sys  # Installing the zipfile module if not already installed
if sys.version_info >= (3, 6):
    import zipfile
else:
    import zipfile36 as zipfile
import os

zip_file_path = '/content/drive/MyDrive/peta_dataset.zip'  # Update with your path
extracted_dir = '/content'  # Where to extract the files

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from tqdm import tqdm
import os

# Define dataset path
DATASET_PATH = "/content/peta_dataset"
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
EPOCHS = 100
PATIENCE = 10  # Early stopping patience
LR = 0.0001
WEIGHT_DECAY = 1e-4  # Regularization

# Define data transformations
transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset using ImageFolder
dataset = ImageFolder(DATASET_PATH, transform=transform)

# Split into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Load Pre-trained EfficientNet
model = models.efficientnet_b0(pretrained=True)
model.classifier = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(model.classifier[1].in_features, 1),
    nn.Sigmoid()
)

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define Loss and Optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

# Early Stopping Implementation
class EarlyStopping:
    def __init__(self, patience=PATIENCE, min_epochs=50, min_accuracy=90.0):
        self.patience = patience
        self.min_epochs = min_epochs
        self.min_accuracy = min_accuracy
        self.best_loss = float("inf")
        self.counter = 0
        self.best_model = None

    def check_early_stop(self, model, val_loss, epoch, val_accuracy):
        if epoch < self.min_epochs or val_accuracy < self.min_accuracy:
            return False  # Continue training
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.counter = 0
            self.best_model = model.state_dict()
        else:
            self.counter += 1
            print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                print("Early stopping triggered.")
                return True
        return False

# Training Function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=EPOCHS):
    early_stopping = EarlyStopping()

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        train_progress = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")

        for images, labels in train_progress:
            images, labels = images.to(device), labels.to(device).float().unsqueeze(1)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            train_progress.set_postfix(loss=running_loss/len(train_loader))

        val_loss, val_accuracy = evaluate_model(model, val_loader)
        scheduler.step(val_loss)

        print(f"Epoch {epoch+1}/{epochs}, Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.2f}%")

        if early_stopping.check_early_stop(model, val_loss, epoch, val_accuracy):
            break

    if early_stopping.best_model:
        model.load_state_dict(early_stopping.best_model)

# Evaluate Model
def evaluate_model(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    val_loss = 0.0
    with torch.no_grad():
        val_progress = tqdm(val_loader, desc="Evaluating")
        for images, labels in val_progress:
            images, labels = images.to(device), labels.to(device).float().unsqueeze(1)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            predicted = (outputs > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return val_loss / len(val_loader), accuracy

# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, lr_scheduler)

# Save Model
torch.save(model.state_dict(), "gender_classification_model.pth")


Evaluating: 100%|██████████| 118/118 [00:19<00:00,  5.94it/s]


Epoch 1/100, Validation Loss: 0.3170, Accuracy: 85.36%


Evaluating: 100%|██████████| 118/118 [00:19<00:00,  5.96it/s]


Epoch 2/100, Validation Loss: 0.2527, Accuracy: 89.53%


Evaluating: 100%|██████████| 118/118 [00:18<00:00,  6.28it/s]


...

Epoch 98/100, Validation Loss: 0.4495, Accuracy: 92.99%


Evaluating: 100%|██████████| 118/118 [00:19<00:00,  5.98it/s]


Epoch 99/100, Validation Loss: 0.4146, Accuracy: 93.15%


Evaluating: 100%|██████████| 118/118 [00:18<00:00,  6.32it/s]

Epoch 100/100, Validation Loss: 0.4587, Accuracy: 93.52%


In [ ]:
# model.load_state_dict(torch.load("gender_classification_model.pth"))
# model.eval()
from PIL import Image

# Function to Predict Gender from Image
def predict_gender(image_path,model):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image)
        predicted = (output[0] > 0.5).float().item()

    return "Male" if predicted == 1 else "Female"

# Example Usage
image_path = "prabhas1.jpg"  # Change this
prediction = predict_gender(image_path,model)
print(f"Predicted Gender: {prediction}")


Predicted Gender: Male


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from tqdm import tqdm
import os

# # Define data transformations
# transform = transforms.Compose([
#     transforms.Resize(IMG_SIZE),
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomRotation(10),
#     transforms.ColorJitter(brightness=0.2, contrast=0.2),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])

# Load Pre-trained EfficientNet
model = models.efficientnet_b0(pretrained=True)
model.classifier = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(model.classifier[1].in_features, 1),
    nn.Sigmoid()
)

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)



/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 122MB/s] 


In [ ]:
import torch
import torch.nn as nn
from torchvision import models

# Recreate the same model architecture
class GenderClassificationModel(nn.Module):
    def __init__(self):
        super(GenderClassificationModel, self).__init__()
        self.model = models.efficientnet_b0(pretrained=True)
        self.model.classifier = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(self.model.classifier[1].in_features, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Initialize model and load state_dict
gender_model = GenderClassificationModel()
# Load the state dictionary into the submodule 'model'
gender_model.model.load_state_dict(torch.load("best_gender_classification_model.pth", map_location=torch.device('cpu')))
gender_model.eval()  # Set to evaluation mode

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
<ipython-input-5-bf6c089c33e1>:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models f

GenderClassificationModel(
  (model): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
          

In [ ]:
torch.save(gender_model, "gender_classification_full.pth")

In [ ]:
final_model = torch.load("gender_classification_full.pth", map_location=torch.device('cpu'))
final_model.eval()  # Set to evaluation mode


<ipython-input-7-edf782983c3b>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  final_model = torch.load("gender_classification_full.pth", map_location=torch.device('cpu'))


GenderClassificationModel(
  (model): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
          

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
# Load the full model
# gender_model = torch.load("gender_classification_full.pth", map_location=torch.device('cpu'))
# gender_model.eval()  # Set to evaluation mode

# Define image preprocessing pipeline
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image to match EfficientNet input size
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNet normalization
])

# Load and preprocess the image
image_path = "prabhas1.jpg"
image = Image.open(image_path).convert("RGB")  # Ensure it's RGB
image_tensor = transform(image).unsqueeze(0)  # Add batch dimension


In [ ]:
def predict_gender(image_path,model):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image)
        predicted = (output[0] > 0.5).float().item()

    return "Male" if predicted == 1 else "Female"

# Example Usage
image_path = "prabhas1.jpg"  # Change this
prediction = predict_gender(image_path,final_model)
print(f"Predicted Gender: {prediction}")

Predicted Gender: Male


In [ ]:
image_path = "/content/disha_patani.jpeg"  # Change this
prediction = predict_gender(image_path,final_model)
print(f"Predicted Gender: {prediction}")

Predicted Gender: Female
